# LIMPIEZA DE DATOS ECOMMERCE

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.graph_objects as go

import sqlalchemy as sa

import re

pd.options.display.float_format = '{:.2f}'.format


%config IPCompleter.greedy=True # Activa el autocompletar avanzado en Jupyter

from IPython.display import display, HTML, Image
display(HTML("""<style> th{ text-align: center !important; font-weight: bold !important; } td{ text-align: center !important; }</style>"""))

---
## ENTENDIENDO EL SIGNIFICADO DE LAS VARIABLES
---

| Variable      	| Tipo de Dato 	| Descripción                                                                             	|
|---------------	|--------------	|-----------------------------------------------------------------------------------------	|
| index         	| BIGINT       	| Índice único de cada fila.                                                              	|
| event_time    	| TEXT         	| Fecha y hora del evento en formato YYYY-MM-DD HH:MM:SS UTC.                             	|
| event_type    	| TEXT         	| Tipo de evento: view (vista de producto), cart (añadido al carrito), purchase (compra). 	|
| product_id    	| BIGINT       	| Identificador único del producto.                                                       	|
| category_id   	| BIGINT       	| Identificador único de la categoría del producto.                                       	|
| category_code 	| TEXT         	| Nombre de la categoría del producto (puede ser NULL).                                   	|
| brand         	| TEXT         	| Nombre de la categoría del producto (puede ser NULL).                                   	|
| price         	| FLOAT        	| Precio del producto en la moneda base.                                                  	|
| user_id       	| BIGINT       	| Identificador único del usuario.                                                        	|
| user_session  	| TEXT         	| Identificador único de la sesión del usuario.                                           	|

---
## IMPORTACION DE DATOS
---

In [3]:
df = pd.read_csv("../data/originals/ecommerce.csv")
df

,index,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,68,2019-10-01 00:01:46 UTC,view,5843665,1487580005092295511,NaN,f.o.x,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361
1,72,2019-10-01 00:01:55 UTC,cart,5868461,1487580013069861041,NaN,italwax,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
2,95,2019-10-01 00:02:50 UTC,view,5877456,1487580006300255120,NaN,jessnail,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770
3,122,2019-10-01 00:03:41 UTC,view,5649270,1487580013749338323,NaN,concept,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df
4,124,2019-10-01 00:03:44 UTC,view,18082,1487580005411062629,NaN,cnd,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5
...,...,...,...,...,...,...,...,...,...,...
2095071,4156660,2020-02-29 23:58:49 UTC,cart,5815662,1487580006317032337,NaN,NaN,0.92,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
2095072,4156663,2020-02-29 23:58:57 UTC,view,5815665,1487580006317032337,NaN,NaN,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
2095073,4156668,2020-02-29 23:59:05 UTC,cart,5815665,1487580006317032337,NaN,NaN,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
2095074,4156675,2020-02-29 23:59:28 UTC,view,5817692,1487580010872045658,NaN,NaN,0.79,619841242,18af673b-7fb9-4202-a66d-5c855bc0fd2d


---
## ANÁLISIS DEL DATAFRAME
---

In [4]:
# Información general del DataFrame
print(f"- Información general del DataFrame:")
df.info()

# Visualizamos la cantidad de filas y columnas
print(f"\n- El DataFrame consta de: {df.shape[0]} filas y {df.shape[1]} columnas")

# Visualizamos el índice del DataFrame
print(f"- El índice del DataFrame es: {df.index}")

# Visualizamos el nombre de las columnas
print(f"- El nombre de las columnas es: {df.columns.tolist()}\n")

- Información general del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2095076 entries, 0 to 2095075
Data columns (total 10 columns):
 #   Column         Dtype  
---  ------         -----  
 0   index          int64  
 1   event_time     object 
 2   event_type     object 
 3   product_id     int64  
 4   category_id    int64  
 5   category_code  object 
 6   brand          object 
 7   price          float64
 8   user_id        int64  
 9   user_session   object 
dtypes: float64(1), int64(4), object(5)
memory usage: 159.8+ MB

- El DataFrame consta de: 2095076 filas y 10 columnas
- El índice del DataFrame es: RangeIndex(start=0, stop=2095076, step=1)
- El nombre de las columnas es: ['index', 'event_time', 'event_type', 'product_id', 'category_id', 'category_code', 'brand', 'price', 'user_id', 'user_session']



In [5]:
# Visualizamos los registros duplicados
print(f"- Registros duplicados: {df.duplicated().sum()}\n")

# Visualizamos los valores unicos de cada columna
print(f"- Valores únicos por columna:\n{df.nunique()}\n")

# Visualizamos los valores nulos por columna
print(f"- Valores nulos por columna:\n{df.isnull().sum()}\n")

- Registros duplicados: 0

- Valores únicos por columna:
index            1731093
event_time       1670797
event_type             4
product_id         46038
category_id          508
category_code         11
brand                266
price               2624
user_id           163936
user_session      448679
dtype: int64

- Valores nulos por columna:
index                  0
event_time             0
event_type             0
product_id             0
category_id            0
category_code    2060411
brand             891646
price                  0
user_id                0
user_session         506
dtype: int64



---
## FORMATEO Y CORRECCIÓN DE NOMBRES DE VARIABLES
---
- La columna index es un indice que viene con los datos que no vamos a utilizar por lo que se puede eliminar
- La columna category_code la podemos eliminar ya que la mayoria de registros son nulos y ya tenemos category_id
- La columna brand (nombre de la categoria del producto) la podemos eliminar ya que tiene muchos nulos, hace referencia al nombre de la categoría del producto y ya utilizamos el category_id
- Formatear y renombrar variables

In [6]:
# Eliminar las columnas 'index', 'category_code' y 'brand'
df = df.drop(columns=['index', 'category_code', 'brand'])

# Limpiar y estandarizar nombres de columnas
df.columns = [re.sub(r'[^\w\s]', '', col).lower().strip(' _').replace(' ', '_') for col in df.columns]

# Renombramos las variables
df.rename(columns={
    'event_time': 'fecha',
    'event_type': 'evento',
    'product_id': 'producto',
    'category_id': 'categoria',
    'price': 'precio',
    'user_id': 'usuario',
    'user_session': 'sesion'
}, inplace=True)
df

,fecha,evento,producto,categoria,precio,usuario,sesion
0,2019-10-01 00:01:46 UTC,view,5843665,1487580005092295511,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361
1,2019-10-01 00:01:55 UTC,cart,5868461,1487580013069861041,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
2,2019-10-01 00:02:50 UTC,view,5877456,1487580006300255120,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770
3,2019-10-01 00:03:41 UTC,view,5649270,1487580013749338323,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df
4,2019-10-01 00:03:44 UTC,view,18082,1487580005411062629,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5
...,...,...,...,...,...,...,...
2095071,2020-02-29 23:58:49 UTC,cart,5815662,1487580006317032337,0.92,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
2095072,2020-02-29 23:58:57 UTC,view,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
2095073,2020-02-29 23:59:05 UTC,cart,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
2095074,2020-02-29 23:59:28 UTC,view,5817692,1487580010872045658,0.79,619841242,18af673b-7fb9-4202-a66d-5c855bc0fd2d


---
## CONVERTIR TIPOS DE DATOS
---
- Convertir 'fecha' en tipo de dato datetime64
- Convertir la variable 'evento' a tipo category ya que tienen pocos valores únicos, esto optimizará memoria y mejorará la velocidad en análisis y modelos de Machine Learning

In [7]:
# Convertir 'fecha' a datetime64 y eliminar el componente de zona horaria (UTC) si existe
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce').dt.tz_localize(None)

# Convertir la variable 'evento' a tipo category
df['evento'] = df['evento'].astype('category')

# Verificar los cambios
df.info()
display(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2095076 entries, 0 to 2095075
Data columns (total 7 columns):
 #   Column     Dtype         
---  ------     -----         
 0   fecha      datetime64[ns]
 1   evento     category      
 2   producto   int64         
 3   categoria  int64         
 4   precio     float64       
 5   usuario    int64         
 6   sesion     object        
dtypes: category(1), datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 97.9+ MB


,fecha,evento,producto,categoria,precio,usuario,sesion
0,2019-10-01 00:01:46,view,5843665,1487580005092295511,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361
1,2019-10-01 00:01:55,cart,5868461,1487580013069861041,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
2,2019-10-01 00:02:50,view,5877456,1487580006300255120,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770
3,2019-10-01 00:03:41,view,5649270,1487580013749338323,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df
4,2019-10-01 00:03:44,view,18082,1487580005411062629,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5
...,...,...,...,...,...,...,...
2095071,2020-02-29 23:58:49,cart,5815662,1487580006317032337,0.92,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
2095072,2020-02-29 23:58:57,view,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
2095073,2020-02-29 23:59:05,cart,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
2095074,2020-02-29 23:59:28,view,5817692,1487580010872045658,0.79,619841242,18af673b-7fb9-4202-a66d-5c855bc0fd2d


---
## ANÁLISIS DE NULOS
---
- 'sesion_usuario' identifica la sesion de navegacion de los usuarios, eliminamos los nulos ya que es una variable relevante y no tiene sentido hacer ningún tipo de imputación

In [8]:
# Visualizamos los valores nulos por columna
print(f"- Valores nulos por columna:\n{df.isnull().sum()}\n")

# Número de registros antes de eliminar nulos
print(f"Registros antes de eliminar nulos: {df.shape[0]}")

# Eliminar los registros con valores nulos (en cualquier columna, pero asumimos que solo 'sesion_usuario' tiene nulos)
df = df.dropna()

# Número de registros después de eliminar nulos
print(f"Registros después de eliminar nulos: {df.shape[0]}")

# Visualizamos los valores nulos por columna
print(f"\n- Valores nulos por columna:\n{df.isnull().sum()}")

- Valores nulos por columna:
fecha          0
evento         0
producto       0
categoria      0
precio         0
usuario        0
sesion       506
dtype: int64

Registros antes de eliminar nulos: 2095076
Registros después de eliminar nulos: 2094570

- Valores nulos por columna:
fecha        0
evento       0
producto     0
categoria    0
precio       0
usuario      0
sesion       0
dtype: int64


---
## ANÁLISIS DE VARIABLES CUANTITATIVAS
---
Conclusiones:

* Existen valores negativos en la variable 'precio', puede ser por devoluciones pero no deberia existir un precio negativo en el producto
* Los registros de los productos donde el precio del valor es menor o igual a cero no se concentran en un solo producto


Acciones:

* eliminar los registros donde el precio del producto es igual o menor a cero

In [9]:
# Número de registros con precio <= 0
print(f"- Cantidad de registros con precio menor o igual a 0: {df[df.precio <= 0].shape[0]}") 

# Frecuencia de los productos con precio <= 0
print(f"\nFrecuencia de los productos con precio <= 0: {df[df.precio <= 0].producto.value_counts().head(10)}")

print(f"\n- Número de registros antes de eliminar precios <= 0: {df.shape[0]}")

# Eliminar los registros donde el precio es igual o menor a cero
df = df[df.precio > 0]

print(f"- Número de registros después de eliminar precios <= 0: {df.shape[0]}")

- Cantidad de registros con precio menor o igual a 0: 20544

Frecuencia de los productos con precio <= 0: producto
5896186    79
5903915    50
5873428    37
5851294    29
5851304    29
5837624    28
5851272    27
5712583    27
5907812    26
5899512    26
Name: count, dtype: int64

- Número de registros antes de eliminar precios <= 0: 2094570
- Número de registros después de eliminar precios <= 0: 2074026


---
## GUARDAR EL DATAFRAME EN FORMATO PICKLE
---

In [10]:
# Guardar el DataFrame en formato pickle
df.to_pickle("../data/processed/ecommerce_cleaning.pkl")
print("El DataFrame ha sido guardado en 'data/processed/' con el nombre 'ecommerce_cleaning.pkl'")

El DataFrame ha sido guardado en 'data/processed/' con el nombre 'ecommerce_cleaning.pkl'


In [12]:
# Guardar el DataFrame en formato CSV
df.to_pickle("../data/originals/ecommerce_cleaning.csv")
print("El DataFrame ha sido guardado en 'data/originals/' con el nombre 'ecommerce_cleaning.csv'")

El DataFrame ha sido guardado en 'data/originals/' con el nombre 'ecommerce_cleaning.csv'
